In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Display all columns without truncation

In [2]:
%%bash
ls train/

train_applprev_1_0.csv
train_applprev_1_1.csv
train_applprev_2.csv
train_base.csv
train_credit_bureau_a_1_0.csv
train_credit_bureau_a_1_1.csv
train_credit_bureau_a_1_2.csv
train_credit_bureau_a_1_3.csv
train_credit_bureau_a_2_0.csv
train_credit_bureau_a_2_10.csv
train_credit_bureau_a_2_1.csv
train_credit_bureau_a_2_2.csv
train_credit_bureau_a_2_3.csv
train_credit_bureau_a_2_4.csv
train_credit_bureau_a_2_5.csv
train_credit_bureau_a_2_6.csv
train_credit_bureau_a_2_7.csv
train_credit_bureau_a_2_8.csv
train_credit_bureau_a_2_9.csv
train_credit_bureau_b_1.csv
train_credit_bureau_b_2.csv
train_debitcard_1.csv
train_deposit_1.csv
train_other_1.csv
train_person_1.csv
train_person_2.csv
train_static_0_0.csv
train_static_0_1.csv
train_static_cb_0.csv
train_tax_registry_a_1.csv
train_tax_registry_b_1.csv
train_tax_registry_c_1.csv


##### We have 32 files to process and merge for the train dataset. This contains all the columns and information that we need for the model. <i> See feature_definition.csv </i>

In [3]:
%%bash
ls train/ | wc -l

32


In [4]:
features = pd.read_csv('feature_definitions.csv')
features.head()

,Variable,Description
0,actualdpd_943P,Days Past Due (DPD) of previous contract (actu...
1,actualdpdtolerance_344P,DPD of client with tolerance.
2,addres_district_368M,District of the person's address.
3,addres_role_871L,Role of person's address.
4,addres_zip_823M,Zip code of the address.


### 0. Base table: Contains format for predictions/actual events
##### train_base.csv contains the case_id's and the target values for each case. This will serve as the base table that contains the actual tags for <i>"likely to default"</i>. Target = 1 means client defaulted, 0 otherwise.

In [5]:
train_base = pd.read_csv('train/train_base.csv', low_memory = False)
train_base.head()

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901.0,0.0,0.0
1,1,2019-01-03,201901.0,0.0,0.0
2,2,2019-01-04,201901.0,0.0,0.0
3,3,2019-01-03,201901.0,0.0,0.0
4,4,2019-01-04,201901.0,0.0,1.0


In [6]:
train_base.describe()

,case_id,MONTH,WEEK_NUM,target
count,1.108969e+06,1.108968e+06,1.108968e+06,1.108968e+06
mean,9.591726e+05,2.019237e+05,3.520375e+01,3.080973e-02
std,5.120217e+05,3.676278e+01,2.122931e+01,1.728019e-01
min,0.000000e+00,2.019010e+05,0.000000e+00,0.000000e+00
25%,6.617750e+05,2.019050e+05,2.000000e+01,0.000000e+00
50%,9.390170e+05,2.019090e+05,3.500000e+01,0.000000e+00
75%,1.425755e+06,2.019120e+05,4.700000e+01,0.000000e+00
max,1.702997e+06,2.020100e+05,9.100000e+01,1.000000e+00


##### Roughly 3.08% of the 1.1M client base defaulted. 

In [7]:
train_base['target'].sum()/len(train_base)

0.0308096980168066

##### test_base.csv is the reference file for prediction. Attach 'target' column (binary 0/1) as predictions

In [8]:
test_base = pd.read_csv('test/test_base.csv', low_memory = False)
test_base.head()

,case_id,date_decision,MONTH,WEEK_NUM
0,57543,2021-05-14,202201,100
1,57549,2022-01-17,202201,100
2,57551,2020-11-27,202201,100
3,57552,2020-11-27,202201,100
4,57569,2021-12-20,202201,100


In [23]:
#Delete tables for more memory
del train_base, test_base

### I. APPLEPREV tables: Contains information on previous applications

In [9]:
train_appl1 = pd.read_csv('train/train_applprev_1_0.csv', low_memory = False)
train_appl1.head()

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_status_367L,credacc_transactions_402L,credamount_590A,credtype_587L,currdebt_94A,dateactivated_425D,district_544M,downpmt_134A,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,firstnonzeroinstldate_307D,inittransactioncode_279L,isbidproduct_390L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,2,0.0,640.2,NaN,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,NaN,NaN,NaN,NaN,10000.0,CAL,NaN,NaN,P136_108_173,0.0,NaN,NaN,P97_36_170,2010-02-15,SINGLE,2013-05-04,CASH,False,NaN,8200.0,NaN,0.0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,24.0
1,2,0.0,1682.4,NaN,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,NaN,NaN,NaN,NaN,16000.0,CAL,NaN,NaN,P136_108_173,0.0,NaN,NaN,P97_36_170,2010-02-15,SINGLE,2013-05-04,CASH,False,NaN,8200.0,NaN,1.0,NaN,12.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,12.0
2,3,0.0,6140.0,NaN,NaN,P94_109_143,NaN,2019-01-07,NaN,0.0,NaN,NaN,NaN,NaN,59999.8,CAL,NaN,NaN,P131_33_167,0.0,NaN,NaN,P97_36_170,2018-05-15,MARRIED,2019-02-07,CASH,False,NaN,11000.0,NaN,0.0,NaN,12.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,12.0
3,4,0.0,2556.6,NaN,NaN,P24_27_36,NaN,2019-01-08,NaN,0.0,NaN,NaN,NaN,NaN,40000.0,CAL,NaN,NaN,P194_82_174,0.0,NaN,NaN,a55475b1,NaN,NaN,2019-02-08,CASH,False,NaN,16000.0,NaN,0.0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,24.0
4,5,0.0,NaN,NaN,NaN,P85_114_140,NaN,2019-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P54_133_26,NaN,NaN,NaN,a55475b1,NaN,NaN,NaN,NaN,False,NaN,62000.0,NaN,0.0,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,NaN


In [10]:
train_appl1.describe()

,case_id,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
count,162601.000000,162559.000000,153437.000000,42766.000000,77158.000000,4056.000000,154452.000000,4056.000000,4056.000000,4056.000000,154300.000000,86801.000000,154299.000000,160915.000000,60786.000000,162600.000000,86587.000000,146245.000000,3.659000e+03,146245.000000
mean,90374.993038,0.001519,3617.830622,20251.759914,0.799334,11983.543961,2424.917630,-3787.521824,-5790.085372,0.652613,45052.386017,7213.986848,245.233219,41316.042497,3.805054,4.433745,10562.259061,18.894116,7.398356e+08,18.894116
std,38469.650760,0.246939,2712.995080,31430.851027,1.324429,21039.361999,12576.140048,15062.957360,16075.048532,4.642046,40346.707728,24372.815646,1852.158732,31491.664854,54.859022,4.554009,38902.010412,11.582598,4.639988e+07,11.582598
min,2.000000,0.000000,0.000000,0.000000,0.000000,-198.194000,0.000000,-102332.300000,-108732.290000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,5.403427e+08,3.000000
25%,103453.000000,0.000000,1920.200100,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18000.000000,0.000000,0.000000,20000.000000,0.000000,1.000000,0.000000,12.000000,7.356110e+08,12.000000
50%,108728.000000,0.000000,3064.400100,2000.000000,0.000000,123.000000,0.000000,0.000000,0.000000,0.000000,32000.000000,0.000000,0.000000,36000.000000,0.000000,3.000000,0.000000,18.000000,7.601459e+08,18.000000
75%,113812.000000,0.000000,4735.000000,30000.000000,1.000000,17998.000000,0.000000,0.000000,0.000000,0.000000,60000.000000,0.000000,0.000000,56000.000000,1.000000,7.000000,0.000000,24.000000,7.604623e+08,24.000000
max,119050.000000,96.000000,105130.200000,200000.000000,15.000000,101408.880000,240000.000000,147011.020000,21225.105000,155.000000,400000.000000,393487.720000,120000.000000,196000.000000,3636.000000,19.000000,873632.440000,60.000000,7.808624e+08,60.000000


In [11]:
train_appl2 = pd.read_csv('train/train_applprev_1_1.csv', low_memory = False)
train_appl2.head()

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_status_367L,credacc_transactions_402L,credamount_590A,credtype_587L,currdebt_94A,dateactivated_425D,district_544M,downpmt_134A,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,firstnonzeroinstldate_307D,inittransactioncode_279L,isbidproduct_390L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,40704,0.0,7204.6000,NaN,NaN,P94_109_143,NaN,2018-11-20,NaN,0.0,NaN,NaN,NaN,NaN,54000.0,CAL,NaN,NaN,P147_6_101,0.0,NaN,NaN,a55475b1,NaN,NaN,2018-12-20,CASH,False,NaN,40000.0,NaN,0,NaN,12.0,P46_145_78,a55475b1,P198_131_9,P94_109_143,NaN,D,12.0
1,40734,0.0,3870.2000,NaN,NaN,P94_109_143,NaN,2019-12-26,NaN,0.0,NaN,NaN,NaN,NaN,50000.0,CAL,NaN,NaN,P111_148_100,0.0,NaN,NaN,a55475b1,NaN,NaN,2020-01-26,CASH,False,NaN,50000.0,NaN,0,NaN,18.0,P149_40_170,a55475b1,P45_84_106,P94_109_143,NaN,D,18.0
2,40737,0.0,2324.4001,NaN,1.0,a55475b1,0.0,2014-07-17,NaN,0.0,NaN,NaN,NaN,NaN,30000.0,CAL,0.0,NaN,a55475b1,0.0,NaN,NaN,P97_36_170,2014-01-15,MARRIED,2014-08-17,CASH,False,NaN,16000.0,NaN,0,0.0,18.0,P46_145_78,a55475b1,a55475b1,a55475b1,NaN,D,18.0
3,40791,0.0,2320.8000,NaN,1.0,a55475b1,0.0,2014-12-28,NaN,0.0,NaN,NaN,NaN,NaN,27830.0,COL,0.0,NaN,a55475b1,0.0,NaN,NaN,P97_36_170,2013-04-15,SINGLE,2015-01-28,POS,False,NaN,16000.0,NaN,1,0.0,12.0,P60_146_156,a55475b1,a55475b1,a55475b1,NaN,D,12.0
4,40791,0.0,2541.2000,NaN,1.0,a55475b1,0.0,2014-12-28,NaN,0.0,NaN,NaN,NaN,NaN,58239.8,COL,0.0,NaN,a55475b1,0.0,NaN,NaN,P97_36_170,2013-04-15,SINGLE,2015-01-28,POS,False,NaN,22000.0,NaN,2,0.0,24.0,P177_117_192,a55475b1,a55475b1,a55475b1,NaN,D,24.0


In [12]:
train_appl2.describe()

,case_id,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
count,160245.000000,160225.000000,153881.000000,31512.000000,56311.000000,6963.000000,154753.000000,6.963000e+03,6963.000000,6963.000000,154549.000000,82002.000000,154549.000000,156553.000000,63400.000000,160245.000000,81833.000000,144980.000000,6.088000e+03,144980.000000
mean,175458.581772,0.213756,3679.973750,20250.527752,0.778622,10357.505679,2928.917006,-1.924399e+03,-5639.776717,0.842597,50210.241681,7917.550299,220.586366,44699.555657,6.960158,5.301507,11562.889845,19.877107,7.623943e+08,19.877107
std,48612.060863,19.629135,2887.580047,31319.746077,1.274480,21678.417772,15343.978607,3.105940e+04,17718.100583,3.730344,50812.985588,26869.851953,1803.351590,32905.300523,88.310286,4.847241,43331.798978,12.526313,4.024378e+07,12.526313
min,40704.000000,0.000000,0.000000,0.000000,0.000000,-20796.000000,0.000000,-1.631820e+05,-169382.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,5.403443e+08,3.000000
25%,188181.000000,0.000000,1872.200100,1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,18396.000000,0.000000,0.000000,22000.000000,0.000000,1.000000,0.000000,12.000000,7.601942e+08,12.000000
50%,192881.000000,0.000000,3055.600000,5000.000000,0.000000,40.000000,0.000000,0.000000e+00,0.000000,0.000000,34000.000000,0.000000,0.000000,39400.000000,0.000000,4.000000,0.000000,18.000000,7.802369e+08,18.000000
75%,197440.000000,0.000000,4869.800300,30000.000000,1.000000,10998.000000,0.000000,2.171800e+01,0.000000,0.000000,60000.000000,0.000000,0.000000,60000.000000,1.000000,8.000000,0.000000,24.000000,7.806384e+08,24.000000
max,201835.000000,4206.000000,81894.600000,200000.000000,20.000000,235200.000000,400000.000000,1.180700e+06,35025.770000,65.000000,1000000.000000,458601.600000,144000.000000,199600.000000,4206.000000,19.000000,866244.000000,62.000000,8.006075e+08,62.000000


In [13]:
#Columns included in applprev tables
features[features.Variable.isin(list(train_appl1.columns))]

,Variable,Description
0,actualdpd_943P,Days Past Due (DPD) of previous contract (actu...
18,annuity_853A,Monthly annuity for previous applications.
26,approvaldate_319D,Approval Date of Previous Application
43,byoccupationinc_3656910L,Applicant's income from previous applications.
45,cancelreason_3545846M,Application cancellation reason.
48,childnum_21L,Number of children in the previous application.
92,creationdate_885D,Date when previous application was created.
93,credacc_actualbalance_314A,Actual balance on credit account.
95,credacc_credlmt_575A,Credit card credit limit provided for previous...
96,credacc_maxhisbal_375A,Maximal historical balance of previous credit ...


In [14]:
#Previous cards status
train_appl3 = pd.read_csv('train/train_applprev_2.csv', low_memory = False)
train_appl3.head()

,case_id,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L,num_group1,num_group2
0,2,NaN,PRIMARY_MOBILE,NaN,0.0,0.0
1,2,NaN,EMPLOYMENT_PHONE,NaN,0.0,1.0
2,2,NaN,PRIMARY_MOBILE,NaN,1.0,0.0
3,2,NaN,EMPLOYMENT_PHONE,NaN,1.0,1.0
4,3,NaN,PHONE,NaN,0.0,0.0


In [15]:
train_appl3.describe()

,case_id,num_group1,num_group2
count,1.032724e+06,1.032723e+06,1.032723e+06
mean,1.237055e+05,5.336385e+00,6.868057e-01
std,3.620237e+04,4.740889e+00,7.999664e-01
min,2.000000e+00,0.000000e+00,0.000000e+00
25%,1.136190e+05,1.000000e+00,0.000000e+00
50%,1.326130e+05,4.000000e+00,0.000000e+00
75%,1.480830e+05,8.000000e+00,1.000000e+00
max,1.627350e+05,1.900000e+01,7.000000e+00


In [16]:
#Columns included in applprev tables
features[features.Variable.isin(list(train_appl3.columns))]

,Variable,Description
44,cacccardblochreas_147M,Card blocking reason.
91,conts_type_509L,Person contact type in previous application.
94,credacc_cards_status_52L,Card status of the previous credit account.


In [17]:
train_appl3.groupby('credacc_cards_status_52L', dropna = False)['case_id'].nunique()

credacc_cards_status_52L
ACTIVE          4807
BLOCKED           71
CANCELLED       6296
INACTIVE        3002
RENEWED           19
UNCONFIRMED        3
NaN            84087
Name: case_id, dtype: int64

In [21]:
# Delete tables to save memory for other tables
del train_appl1, train_appl2, train_appl3

In [22]:
[key for key, value in globals().items() if not key.startswith('_') and not callable(value)]

['In', 'Out', 'pd', 'features', 'train_base', 'test_base']

### II. Credit Bureau Data: Contains multisource data from the Credit Bureau on a consumer's historical credit and financial data

In [33]:
# 79-column data: all same format for these set of files
train_credita10 = pd.read_csv('train/train_credit_bureau_a_1_0.csv', low_memory = False)
train_credita11 = pd.read_csv('train/train_credit_bureau_a_1_1.csv', low_memory = False)
train_credita12 = pd.read_csv('train/train_credit_bureau_a_1_2.csv', low_memory = False)
train_credita13 = pd.read_csv('train/train_credit_bureau_a_1_3.csv', low_memory = False)

In [34]:
train_credita10.head()

,case_id,annualeffectiverate_199L,annualeffectiverate_63L,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,contractsum_5085717L,credlmt_230A,credlmt_935A,dateofcredend_289D,dateofcredend_353D,dateofcredstart_181D,dateofcredstart_739D,dateofrealrepmt_138D,debtoutstand_525A,debtoverdue_47A,description_351M,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,financialinstitution_382M,financialinstitution_591M,instlamount_768A,instlamount_852A,interestrate_508L,lastupdate_1112D,lastupdate_388D,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,num_group1,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,numberofoutstandinstls_59L,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstlmaxdat_148D,numberofoverdueinstlmaxdat_641D,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax2date_1002D,overdueamountmax2date_1142D,overdueamountmax_155A,overdueamountmax_35A,overdueamountmaxdatemonth_284T,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,purposeofcred_426M,purposeofcred_874M,refreshdate_3813885D,residualamount_488A,residualamount_856A,subjectrole_182M,subjectrole_93M,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
0,388,NaN,NaN,ea6782cc,a55475b1,7241344e,a55475b1,NaN,NaN,135806.0,2020-08-06,NaN,NaN,2018-08-06,NaN,NaN,NaN,a55475b1,0.0,NaN,NaN,8.0,2018.0,NaN,a55475b1,P204_66_73,8742.8,NaN,NaN,2019-01-11,NaN,8742.8000,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,8.0,2018.0,NaN,NaN,NaN,NaN,NaN,60c73645,a55475b1,NaN,NaN,114325.805,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN
1,388,NaN,NaN,4408ff0f,a55475b1,7241344e,a55475b1,NaN,NaN,NaN,2023-06-20,NaN,NaN,2018-06-20,NaN,374419.5,0.0,a55475b1,0.0,NaN,NaN,7.0,2018.0,NaN,a55475b1,55b002a9,NaN,NaN,NaN,2019-01-24,NaN,7811.4463,NaN,NaN,NaN,0,2.0,1.0,NaN,60.0,NaN,54.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,260093.7,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,7.0,2018.0,NaN,NaN,30.0,NaN,NaN,96a8fdfe,a55475b1,NaN,NaN,NaN,ab3c25cf,ab3c25cf,NaN,268897.62,0.0,0.0,374419.5,0.0
2,388,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,2019-01-28,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN
3,388,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,2019-01-28,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN
4,388,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,2019-01-28,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
train_credita10.describe()

,case_id,annualeffectiverate_199L,annualeffectiverate_63L,contractsum_5085717L,credlmt_230A,credlmt_935A,debtoutstand_525A,debtoverdue_47A,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,instlamount_768A,instlamount_852A,interestrate_508L,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,num_group1,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,numberofoutstandinstls_59L,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax_155A,overdueamountmax_35A,overdueamountmaxdatemonth_284T,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,residualamount_488A,residualamount_856A,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
count,159696.000000,922.000000,3514.000000,0.0,1466.000000,1.197600e+04,1.401800e+04,1.401800e+04,22652.000000,8130.000000,8130.000000,22652.000000,22652.000000,8130.000000,11854.000000,915.000000,135.000000,22637.000000,7.549000e+03,4182.000000,859.000000,159696.000000,14000.000000,9820.000000,6944.000000,10783.000000,6953.000000,10783.000000,22768.000000,8431.000000,22648.000000,8401.000000,6954.000000,1.079100e+04,8404.0,2.264900e+04,2.276800e+04,8.431000e+03,2.276800e+04,8.156000e+03,8156.00000,22768.000000,22768.000000,8156.000000,5904.000000,10356.000000,852.000000,383.000000,1454.0,1.185400e+04,6.969000e+03,1.079200e+04,1.400000e+04,9820.000000,1.400000e+04,9820.000000
mean,13500.811417,1387.780456,146.583873,NaN,25627.634270,7.817407e+04,1.285745e+05,4.728320e+02,10.467773,76.650677,6.428536,6.286862,2017.950380,2013.881673,3995.350472,797.682630,22.077926,5262.578587,4.700366e+03,38.996817,212.770105,5.391099,1.617500,4.024440,11.083093,29.560976,0.071911,20.341927,13.689037,88.258925,3.913546,0.000952,1.816370,1.278141e+05,0.0,2.926490e+02,2.377837e+03,8.872226e+03,1.843848e+03,7.985660e+03,6.46665,6.302574,2017.942595,2013.813879,30.209858,30.593955,0.460094,0.647520,0.0,3.531427e+04,1.339337e+05,1.698887e+05,4.734433e+02,92.673279,1.284184e+05,45.766049
std,17489.370903,8671.468555,2721.997114,NaN,44057.606412,7.622456e+05,3.797660e+05,1.647323e+04,109.290628,327.682877,3.338141,3.412083,0.723546,4.035245,5166.784618,2014.381639,7.247210,10836.158109,3.765196e+04,204.278363,571.805437,3.895578,0.828829,4.135698,20.508300,28.786796,1.973576,24.299796,122.973751,373.134272,80.993949,0.077146,109.710063,3.295394e+05,0.0,1.286699e+04,2.036577e+04,7.562709e+04,1.867143e+04,6.827265e+04,3.37793,3.415686,0.720901,3.982911,6.898369,9.081357,1.401545,3.038761,0.0,7.096092e+04,1.974535e+06,4.115831e+05,1.648381e+04,5129.103713,3.680197e+05,4047.107267
min,388.000000,0.000000,0.000000,NaN,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-1.000000,1.000000,1.000000,2015.000000,2005.000000,0.000000,0.000000,8.100000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.00000,1.000000,2015.000000,2005.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,1.000000e+03,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,5744.000000,19.750000,9.115000,NaN,0.000000,0.000000e+00,1.110663e+04,0.000000e+00,0.000000,0.000000,4.000000,3.000000,2017.000000,2011.000000,0.000000,0.000000,18.500000,1465.000000,0.000000e+00,18.000000,24.000000,2.000000,1.000000,1.000000,0.000000,12.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.61

In [41]:
print(f"Total Columns of first file: {len(train_credita10.columns)}")
print(f"First vs 2nd File: {sum(train_credita10.columns == train_credita11.columns)} match")
print(f"First vs 3rd File: {sum(train_credita10.columns == train_credita12.columns)} match")
print(f"First vs 4th File: {sum(train_credita10.columns == train_credita13.columns)} match")

Total Columns of first file: 79
First vs 2nd File: 79 match
First vs 3rd File: 79 match
First vs 4th File: 79 match


In [42]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_credita10.columns)])

,Variable,Description
15,annualeffectiverate_199L,Interest rate of the closed contracts.
16,annualeffectiverate_63L,Interest rate for the active contracts.
50,classificationofcontr_13M,Classificiation of the active contract.
51,classificationofcontr_400M,Classificiation of the closed contract.
86,contractst_545M,Contract status.
87,contractst_964M,Contract status of terminated credit contract.
88,contractsum_5085717L,Sum of other contract values.
104,credlmt_230A,Credit limit of the closed credit contracts fr...
106,credlmt_935A,Credit limit for active loan.
121,dateofcredend_289D,End date of an active credit contract.


In [53]:
# 19-column data: all same format for these set of files
train_credita20 = pd.read_csv('train/train_credit_bureau_a_2_0.csv', low_memory = False)
train_credita21 = pd.read_csv('train/train_credit_bureau_a_2_1.csv', low_memory = False)
train_credita210 = pd.read_csv('train/train_credit_bureau_a_2_10.csv', low_memory = False)
train_credita22 = pd.read_csv('train/train_credit_bureau_a_2_2.csv', low_memory = False)
train_credita23 = pd.read_csv('train/train_credit_bureau_a_2_3.csv', low_memory = False)
train_credita24 = pd.read_csv('train/train_credit_bureau_a_2_4.csv', low_memory = False)
train_credita25 = pd.read_csv('train/train_credit_bureau_a_2_5.csv', low_memory = False)
train_credita26 = pd.read_csv('train/train_credit_bureau_a_2_6.csv', low_memory = False)
train_credita27 = pd.read_csv('train/train_credit_bureau_a_2_7.csv', low_memory = False)
train_credita28 = pd.read_csv('train/train_credit_bureau_a_2_8.csv', low_memory = False)
train_credita29 = pd.read_csv('train/train_credit_bureau_a_2_9.csv', low_memory = False)

In [54]:
train_credita20.head()

,case_id,collater_typofvalofguarant_298M,collater_typofvalofguarant_407M,collater_valueofguarantee_1124L,collater_valueofguarantee_876L,collaterals_typeofguarante_359M,collaterals_typeofguarante_669M,num_group1,num_group2,pmts_dpd_1073P,pmts_dpd_303P,pmts_month_158T,pmts_month_706T,pmts_overdue_1140A,pmts_overdue_1152A,pmts_year_1139T,pmts_year_507T,subjectroles_name_541M,subjectroles_name_838M
0,388,8fd95e4b,a55475b1,0.0,NaN,a55475b1,c7a5ad39,0,0,NaN,NaN,2.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,ab3c25cf
1,388,9a0c095e,a55475b1,0.0,NaN,a55475b1,c7a5ad39,1,0,NaN,NaN,2.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,ab3c25cf
2,388,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,0,1,NaN,NaN,3.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,a55475b1
3,388,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,0,2,NaN,NaN,4.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,a55475b1
4,388,a55475b1,a55475b1,NaN,NaN,a55475b1,a55475b1,0,3,NaN,NaN,5.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,a55475b1


In [55]:
train_credita20.describe()

,case_id,collater_valueofguarantee_1124L,collater_valueofguarantee_876L,num_group1,num_group2,pmts_dpd_1073P,pmts_dpd_303P,pmts_month_158T,pmts_month_706T,pmts_overdue_1140A,pmts_overdue_1152A,pmts_year_1139T,pmts_year_507T
count,385506.000000,1.565500e+04,0.0,385506.000000,385506.000000,191410.000000,0.0,385506.000000,0.0,1.915070e+05,0.0,385506.000000,0.0
mean,52575.416896,3.039397e+05,NaN,0.673110,13.341574,7.305773,NaN,6.499969,NaN,7.259622e+02,NaN,2018.217602,NaN
std,50866.025679,2.129168e+06,NaN,0.966944,9.076101,98.672917,NaN,3.452046,NaN,1.432653e+04,NaN,0.768913,NaN
min,388.000000,0.000000e+00,NaN,0.000000,0.000000,0.000000,NaN,1.000000,NaN,0.000000e+00,NaN,2015.000000,NaN
25%,3944.000000,0.000000e+00,NaN,0.000000,6.000000,0.000000,NaN,3.250000,NaN,0.000000e+00,NaN,2018.000000,NaN
50%,6747.000000,0.000000e+00,NaN,0.000000,12.000000,0.000000,NaN,6.000000,NaN,0.000000e+00,NaN,2018.000000,NaN
75%,105782.000000,0.000000e+00,NaN,1.000000,20.000000,0.000000,NaN,9.000000,NaN,0.000000e+00,NaN,2019.000000,NaN
max,107501.000000,9.983199e+07,NaN,10.000000,35.000000,3513.000000,NaN,12.000000,NaN,1.144263e+06,NaN,2020.000000,NaN


In [56]:
print(f"Total Columns of first file: {len(train_credita20.columns)}")
print(f"First vs 2nd File: {sum(train_credita20.columns == train_credita21.columns)} match")
print(f"First vs 3rd File: {sum(train_credita20.columns == train_credita210.columns)} match")
print(f"First vs 4th File: {sum(train_credita20.columns == train_credita22.columns)} match")
print(f"First vs 5th File: {sum(train_credita20.columns == train_credita23.columns)} match")
print(f"First vs 6th File: {sum(train_credita20.columns == train_credita24.columns)} match")
print(f"First vs 7th File: {sum(train_credita20.columns == train_credita25.columns)} match")
print(f"First vs 8th File: {sum(train_credita20.columns == train_credita26.columns)} match")
print(f"First vs 9th File: {sum(train_credita20.columns == train_credita27.columns)} match")
print(f"First vs 10th File: {sum(train_credita20.columns == train_credita28.columns)} match")
print(f"First vs 11th File: {sum(train_credita20.columns == train_credita29.columns)} match")

Total Columns of first file: 19
First vs 2nd File: 19 match
First vs 3rd File: 19 match
First vs 4th File: 19 match
First vs 5th File: 19 match
First vs 6th File: 19 match
First vs 7th File: 19 match
First vs 8th File: 19 match
First vs 9th File: 19 match
First vs 10th File: 19 match
First vs 11th File: 19 match


In [57]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_credita20.columns)])

,Variable,Description
70,collater_typofvalofguarant_298M,Collateral valuation type (active contract).
71,collater_typofvalofguarant_407M,Collateral valuation type (closed contract).
72,collater_valueofguarantee_1124L,Value of collateral for active contract.
73,collater_valueofguarantee_876L,Value of collateral for closed contract.
74,collaterals_typeofguarante_359M,Type of collateral that was used as a guarante...
75,collaterals_typeofguarante_669M,Collateral type for the active contract.
384,pmts_dpd_1073P,Days past due of the payment for the active co...
385,pmts_dpd_303P,Days past due of the payment for terminated co...
387,pmts_month_158T,Month of payment for a closed contract (num_gr...
388,pmts_month_706T,Month of payment for active contract (num_grou...


In [72]:
#Delete files to save memory
df_items = [key for key, value in globals().items() if key.startswith('train_') and not callable(value)]
for key in df_items:
    del globals()[key]
del df_items, value, key

In [74]:
# 45-column data: standalone file
train_creditb1 = pd.read_csv('train/train_credit_bureau_b_1.csv', low_memory = False)

In [75]:
train_creditb1.head()

,case_id,amount_1115A,classificationofcontr_1114M,contractdate_551D,contractmaturitydate_151D,contractst_516M,contracttype_653M,credlmt_1052A,credlmt_228A,credlmt_3940954A,credor_3940957M,credquantity_1099L,credquantity_984L,debtpastduevalue_732A,debtvalue_227A,dpd_550P,dpd_733P,dpdmax_851P,dpdmaxdatemonth_804T,dpdmaxdateyear_742T,installmentamount_644A,installmentamount_833A,instlamount_892A,interesteffectiverate_369L,interestrateyearly_538L,lastupdate_260D,maxdebtpduevalodued_3940955A,num_group1,numberofinstls_810L,overdueamountmax_950A,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,periodicityofpmts_997L,periodicityofpmts_997M,pmtdaysoverdue_1135P,pmtmethod_731M,pmtnumpending_403L,purposeofcred_722M,residualamount_1093A,residualamount_127A,residualamount_3940956A,subjectrole_326M,subjectrole_43M,totalamount_503A,totalamount_881A
0,467,78000.0,ea6782cc,2016-10-25,2019-10-25,7241344e,4257cbed,NaN,NaN,NaN,c5a72b57,NaN,NaN,0.0,26571.969,NaN,NaN,0.0,11.0,2016.0,NaN,NaN,2898.760,NaN,NaN,2019-01-10,0.0,2,36.0,0.0,11.0,2016.0,NaN,a0b598e4,0.0,e914c86c,10.0,96a8fdfe,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN
1,467,NaN,ea6782cc,2011-06-15,2031-06-13,7241344e,724be82a,3000000.0,10000.0,3000000.0,P164_34_168,2.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.000,NaN,NaN,NaN,2019-01-20,NaN,0,NaN,NaN,NaN,NaN,NaN,a55475b1,NaN,a55475b1,NaN,96a8fdfe,0.0,0.0,NaN,fa4f56f1,ab3c25cf,3000000.0,10000.0
2,467,NaN,ea6782cc,2019-01-04,2021-08-04,7241344e,724be82a,NaN,NaN,130365.0,P164_34_168,1.0,2.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,26571.969,NaN,NaN,NaN,2019-01-20,NaN,1,NaN,NaN,NaN,NaN,NaN,a55475b1,NaN,a55475b1,NaN,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,78000.0,960000.0
3,1445,12000.0,ea6782cc,2018-12-31,2019-01-29,7241344e,4257cbed,NaN,NaN,NaN,0aebc0bb,NaN,NaN,0.0,19066.640,NaN,NaN,0.0,1.0,2019.0,NaN,NaN,19571.412,NaN,NaN,2019-01-27,0.0,2,1.0,0.0,1.0,2019.0,NaN,d479a207,0.0,dbcbe8f8,1.0,96a8fdfe,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN
4,1445,31400.0,01f63ac8,2018-07-25,2019-12-25,7241344e,4257cbed,NaN,NaN,NaN,50babcd4,NaN,NaN,0.0,23390.160,NaN,NaN,0.0,8.0,2018.0,NaN,NaN,2124.142,NaN,NaN,2019-01-28,0.0,3,17.0,0.0,8.0,2018.0,NaN,a0b598e4,0.0,dbcbe8f8,12.0,60c73645,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN


In [76]:
train_creditb1.describe()

,case_id,amount_1115A,credlmt_1052A,credlmt_228A,credlmt_3940954A,credquantity_1099L,credquantity_984L,debtpastduevalue_732A,debtvalue_227A,dpd_550P,dpd_733P,dpdmax_851P,dpdmaxdatemonth_804T,dpdmaxdateyear_742T,installmentamount_644A,installmentamount_833A,instlamount_892A,interesteffectiverate_369L,interestrateyearly_538L,maxdebtpduevalodued_3940955A,num_group1,numberofinstls_810L,overdueamountmax_950A,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,pmtdaysoverdue_1135P,pmtnumpending_403L,residualamount_1093A,residualamount_127A,residualamount_3940956A,totalamount_503A,totalamount_881A
count,8.579100e+04,4.368100e+04,2.758100e+04,1.613000e+04,3.821800e+04,53018.00000,46228.000000,8.121700e+04,4.368100e+04,5.301800e+04,4.622800e+04,8.122400e+04,81224.000000,81224.000000,4.622800e+04,5.301800e+04,4.349300e+04,9506.000000,28825.000000,81224.000000,85791.000000,43493.000000,81224.000000,81224.000000,81224.000000,81217.000000,43680.000000,16125.000000,2.758100e+04,3.751900e+04,5.301800e+04,4.622800e+04
mean,1.218998e+06,2.141105e+05,1.789356e+05,5.231752e+04,1.303603e+05,1.54474,4.565220,3.791878e+03,1.651183e+05,2.569675e+04,3.722601e+02,3.537896e+04,6.610263,2018.226226,3.476580e+02,1.568889e+05,9.814833e+03,504.612400,52.493346,15.074052,1.011749,30.496264,18.251589,6.630676,2018.290825,71.478619,20.264675,0.019969,5.794230e+04,4.301107e+04,2.570089e+05,2.937632e+05
std,6.863320e+05,6.910196e+05,5.274022e+06,1.280825e+05,2.570305e+06,0.86419,5.170388,1.582381e+05,5.504971e+05,9.686434e+05,7.187137e+04,8.772968e+05,3.485599,1.783528,5.411026e+04,6.266627e+05,2.119738e+05,5337.928146,636.047210,1036.106798,1.234288,36.060070,1036.772496,3.494757,1.741405,4664.511050,29.017417,2.535748,1.106215e+05,9.314620e+04,3.876513e+06,1.202831e+06
min,4.670000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.00000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1900.000000,0.000000e+00,0.000000e+00,0.000000e+00,-1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1900.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.272010e+05,2.599800e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.00000,1.000000,0.000000e+00,1.294600e+04,0.000000e+00,0.000000e+00,0.000000e+00,4.000000,2018.000000,0.000000e+00,6.116500e+03,2.372600e+03,5.350000,7.000000,0.000000,0.000000,12.000000,0.000000,4.000000,2018.000000,0.000000,5.000000,0.000000,0.000000e+00,0.000000e+00,2.000000e+04,2.572170e+04
50%,1.413976e+06,6.000000e+04,3.618400e+04,2.260000e+04,2.000000e+04,1.00000,3.000000,0.000000e+00,3.589341e+04,0.000000e+00,0.000000e+00,0.000000e+00,7.000000,2019.000000,0.000000e+00,3.626439e+04,4.228200e+03,23.550000,39.000000,0.000000,1.000000,22.000000,0.000000,7.000000,2019.000000,0.000000,11.000000,0.000000,1.446133e+04,7.483786e+03,6.837960e+04,8.444100e+04
75%,1.778253e+06,1.600000e+05,1.217780e+05,6.000000e+04,7.800000e+04,2.00000,6.000000,0.000000e+00,1.108016e+05,0.000000e+00,0.000000e+00,7.073500e+03,10.000000,2019.000000,0.000000e+00,1.285804e+05,7.804800e+03,40.590000,42.000000,0.200000,2.000000,36.000000,0.400000,10.000000,2019.000000,0.000000,24.000000,0.000000,7.092540e+04,4.132062e+04,2.000000e+05,2.761022e+05
max,2.703436e+06,5.483333e+07,7.968000e+08,4.420000e+06,3.000000e+08,16.00000,146.000000,4.113871e+07,4.161905e+07,2.078238e+08,1.544394e+07,1.851242e+08,12.000000,2020.000000,1.141860e+07,6.965854e+07,4.113871e+07,73000.000000,46334.100000,147470.610000,20.000000,358.000000,147470.610000,12.000000,2020.000000,663618.000000,300.000000,322.000000,2.187568e+06,2.022909e+06,7.968000e+08,1.390800e+08


In [78]:
train_creditb1.shape

(85791, 45)

In [79]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_creditb1.columns)])

,Variable,Description
5,amount_1115A,Credit amount of the active contract provided ...
49,classificationofcontr_1114M,Classificiation of the active contract.
81,contractdate_551D,Contract date of the active contract
83,contractmaturitydate_151D,End date of active contract.
85,contractst_516M,Contract status.
89,contracttype_653M,Contract Type
102,credlmt_1052A,Credit limit of an active loan.
103,credlmt_228A,Credit limit for closed loans.
105,credlmt_3940954A,Credit limit for active loan.
107,credor_3940957M,Creditor's name


In [81]:
# 6-column data: standaline file
train_creditb2 = pd.read_csv('train/train_credit_bureau_b_2.csv', low_memory = False)
train_creditb2.head()

,case_id,num_group1,num_group2,pmts_date_1107D,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
0,467,0,0.0,2018-11-15,NaN,NaN
1,467,0,1.0,2018-12-15,NaN,NaN
2,467,1,0.0,2018-12-15,NaN,NaN
3,467,2,0.0,2016-10-15,0.0,0.0
4,467,2,1.0,2016-11-15,0.0,0.0


In [82]:
train_creditb2.describe()

,case_id,num_group1,num_group2,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
count,1.126829e+06,1.126829e+06,1.126828e+06,1.122219e+06,1.122219e+06
mean,1.101951e+06,7.402862e-01,1.234354e+01,2.606544e+04,1.294303e+01
std,6.190147e+05,1.121750e+00,1.004560e+01,6.127052e+05,4.861975e+02
min,4.670000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.984890e+05,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00
50%,1.336996e+06,0.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00
75%,1.668404e+06,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00
max,1.886627e+06,2.000000e+01,3.600000e+01,1.851242e+08,1.474706e+05


In [83]:
train_creditb2.shape

(1126829, 6)

In [84]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_creditb2.columns)])

,Variable,Description
383,pmts_date_1107D,Payment date for an active contract according ...
386,pmts_dpdvalue_108P,Value of past due payment for active contract ...
391,pmts_pmtsoverdue_635A,Active contract that has overdue payments (num...


In [87]:
#Delete files to save memory
df_items = [key for key, value in globals().items() if key.startswith('train_') and not callable(value)]
for key in df_items:
    del globals()[key]
del df_items, key

### III. Debit Card Data

In [88]:
# 6-column data: standalone file
train_debit1 = pd.read_csv('train/train_debitcard_1.csv', low_memory = False)
train_debit1.head()

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D
0,225,NaN,NaN,NaN,0,2016-08-16
1,331,NaN,NaN,NaN,0,2015-03-19
2,358,NaN,NaN,NaN,0,2014-09-02
3,390,NaN,NaN,NaN,0,2014-07-23
4,390,NaN,NaN,NaN,1,2015-10-01


In [89]:
train_debit1.describe()

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1
count,1.573020e+05,12216.000000,1.108100e+04,11081.000000,157302.000000
mean,1.468784e+06,109.635884,3.849451e+04,4955.383495,0.549306
std,8.883316e+05,949.997458,4.140059e+04,19217.736947,1.639082
min,2.250000e+02,-308.794130,-1.877800e+05,-477.506000,0.000000
25%,6.491730e+05,0.000000,7.878000e+03,0.000000,0.000000
50%,1.560121e+06,0.000000,3.000000e+04,0.000000,0.000000
75%,2.531590e+06,1.053898,6.000000e+04,0.000000,1.000000
max,2.703453e+06,67777.770000,1.161820e+06,390000.000000,65.000000


In [90]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_debit1.columns)])

,Variable,Description
226,last180dayaveragebalance_704A,Average balance on debit card in the last 180 ...
227,last180dayturnover_1134A,Debit card's turnover within the last 180 days.
228,last30dayturnover_651A,Debit card turnover for the last 30 days.
337,openingdate_857D,Debit card opening date.


In [91]:
#Delete files to save memory
df_items = [key for key, value in globals().items() if key.startswith('train_') and not callable(value)]
for key in df_items:
    del globals()[key]
del df_items, key

### IV. Deposit Data

In [92]:
# 5-column data: standalone file
train_deposit1 = pd.read_csv('train/train_deposit_1.csv', low_memory = False)
train_deposit1.head()

,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
0,225,0.00000,NaN,0,2016-08-16
1,331,260.37400,2018-03-18,0,2015-03-19
2,358,0.00000,NaN,0,2014-09-02
3,390,203.60200,2017-09-30,1,2015-10-01
4,390,223.68001,NaN,2,2016-06-08


In [93]:
train_deposit1.describe()

,case_id,amount_416A,num_group1
count,1.450860e+05,1.450860e+05,145086.000000
mean,1.466214e+06,8.422304e+03,0.522531
std,8.865290e+05,8.623212e+04,1.620954
min,2.250000e+02,-4.000000e+04,0.000000
25%,6.600410e+05,0.000000e+00,0.000000
50%,1.556939e+06,2.236580e+02,0.000000
75%,2.530539e+06,4.783400e+02,1.000000
max,2.703453e+06,1.221329e+07,64.000000


In [94]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_deposit1.columns)])

,Variable,Description
6,amount_416A,Deposit amount.
82,contractenddate_991D,End date of deposit contract.
336,openingdate_313D,Deposit account opening date.


### V. Other Data

In [95]:
# 7-column data: standalone file
train_other1 = pd.read_csv('train/train_other_1.csv', low_memory = False)
train_other1.head()

,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,num_group1
0,43801,12466.6010,12291.2000,914.2000,0.0,304.80002,0
1,43991,3333.4001,3273.4001,0.0000,0.0,0.00000,0
2,44001,10000.0000,10000.0000,0.0000,0.0,0.00000,0
3,44053,0.0000,0.0000,2586.4001,0.0,88.80000,0
4,44130,63.8000,60.8000,0.0000,0.0,0.00000,0


In [96]:
train_other1.describe()

,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,num_group1
count,5.110900e+04,5.110900e+04,5.110900e+04,5.110900e+04,5.110900e+04,5.110900e+04,51109.0
mean,1.419514e+06,7.552902e+03,7.462384e+03,9.967413e+03,2.949396e+03,3.586875e+03,0.0
std,9.245095e+05,3.462571e+04,3.506529e+04,8.939342e+04,4.146773e+04,4.827494e+04,0.0
min,4.380100e+04,0.000000e+00,0.000000e+00,-3.357180e+05,0.000000e+00,0.000000e+00,0.0
25%,2.422410e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,1.811468e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.800000e+00,0.0
75%,1.916206e+06,8.000000e+03,7.740000e+03,2.880000e+02,0.000000e+00,5.400000e+00,0.0
max,2.703453e+06,4.957852e+06,5.168004e+06,4.256314e+06,4.180150e+06,4.622918e+06,0.0


In [97]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_other1.columns)])

,Variable,Description
9,amtdebitincoming_4809443A,Incoming debit card transactions amount.
10,amtdebitoutgoing_4809440A,Outgoing debit card transactions amount.
11,amtdepositbalance_4809441A,Deposit balance of client.
12,amtdepositincoming_4809444A,Amount of incoming deposits to client's account.
13,amtdepositoutgoing_4809442A,Amount of outgoing deposits from client's acco...


In [100]:
#Delete files to save memory
df_items = [key for key, value in globals().items() if key.startswith('train_') and not callable(value)]
for key in df_items:
    del globals()[key]
del df_items, key

### VI. Person Data

In [106]:
# 37-column data: standalone file
train_person1 = pd.read_csv('train/train_person_1.csv', low_memory = False)
train_person1.head()

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,gender_992L,housetype_905L,housingtype_772L,incometype_1044T,isreference_387L,language1_981M,mainoccupationinc_384A,maritalst_703L,num_group1,personindex_1023L,persontype_1072L,persontype_792L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,NaN,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,MORE_FIVE,OTHER,P142_57_166,P167_100_165,MARRIED,NaN,NaN,NaN,SALARIED_GOVT,NaN,P10_39_147,10800.0,NaN,0,0.0,1.0,1.0,P88_18_84,P167_100_165,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
1,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,NaN,NaN,1,1.0,1.0,4.0,a55475b1,a55475b1,SPOUSE,NaN,False,EM,NaN,NaN,NaN,PHONE
2,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,NaN,NaN,2,2.0,4.0,5.0,a55475b1,a55475b1,COLLEAGUE,SPOUSE,False,PE,NaN,NaN,NaN,PHONE
3,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,NaN,NaN,3,NaN,5.0,NaN,a55475b1,a55475b1,NaN,COLLEAGUE,NaN,PE,NaN,NaN,NaN,PHONE
4,1,1957-08-01,NaN,NaN,P103_93_94,False,False,P176_37_166,P97_36_170,2008-10-29,MORE_FIVE,OTHER,P49_46_174,P160_59_140,DIVORCED,NaN,NaN,NaN,SALARIED_GOVT,NaN,P10_39_147,10000.0,NaN,0,0.0,1.0,1.0,P103_93_94,P176_37_166,NaN,NaN,NaN,CL,NaN,True,M,PRIMARY_MOBILE


In [107]:
train_person1.shape

(220362, 37)

In [108]:
train_person1.describe()

,case_id,childnum_185L,mainoccupationinc_384A,num_group1,personindex_1023L,persontype_1072L,persontype_792L
count,220362.000000,411.000000,68755.000000,220362.000000,169955.000000,220339.000000,169955.000000
mean,38044.439849,0.513382,57740.427221,1.270528,0.824736,2.783897,3.306622
std,30768.424601,0.919539,35494.124934,1.121288,0.782555,1.944024,1.918450
min,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000
25%,15813.000000,0.000000,32000.000000,0.000000,0.000000,1.000000,1.000000
50%,31943.000000,0.000000,50000.000000,1.000000,1.000000,1.000000,5.000000
75%,48020.000000,1.000000,71400.000000,2.000000,1.000000,5.000000,5.000000
max,114188.000000,7.000000,200000.000000,7.000000,5.000000,5.000000,5.000000


In [109]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_person1.columns)])

,Variable,Description
40,birth_259D,Date of birth of the person.
42,birthdate_87D,Birth date of the person.
47,childnum_185L,Number of children of the applicant.
77,contaddr_district_15M,Zip code of a contact person's address.
78,contaddr_matchlist_1032L,Indicates whether the contact address is found...
79,contaddr_smempladdr_334L,Indicates whether the contact address is the s...
80,contaddr_zipcode_807M,Zip code of contact address.
162,education_927M,Education level of the person.
164,empl_employedfrom_271D,Start date of employment.
165,empl_employedtotal_800L,Employment length of a person.


In [110]:
# 11-column data: standalone file
train_person2 = pd.read_csv('train/train_person_2.csv', low_memory = False)
train_person2.head()

,case_id,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,num_group1,num_group2,relatedpersons_role_762T
0,5,a55475b1,NaN,a55475b1,a55475b1,a55475b1,NaN,a55475b1,0.0,0.0,NaN
1,6,P55_110_32,CONTACT,P10_68_40,P38_92_157,P164_110_33,NaN,a55475b1,0.0,0.0,NaN
2,6,P55_110_32,PERMANENT,P10_68_40,a55475b1,a55475b1,NaN,a55475b1,0.0,1.0,NaN
3,6,P204_92_178,CONTACT,P65_136_169,P38_92_157,P164_110_33,NaN,a55475b1,1.0,0.0,OTHER_RELATIVE
4,6,P191_109_75,CONTACT,P10_68_40,P7_147_157,a55475b1,NaN,a55475b1,1.0,1.0,OTHER_RELATIVE


In [113]:
train_person2.describe()

,case_id,num_group1,num_group2
count,597954.000000,597953.000000,597953.0000
mean,543806.775158,0.171375,0.1026
std,305391.081332,0.390249,0.6901
min,5.000000,0.000000,0.0000
25%,194741.250000,0.000000,0.0000
50%,674319.500000,0.000000,0.0000
75%,791150.750000,0.000000,0.0000
max,906985.000000,3.000000,31.0000


In [114]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_person2.columns)])

,Variable,Description
2,addres_district_368M,District of the person's address.
3,addres_role_871L,Role of person's address.
4,addres_zip_823M,Zip code of the address.
90,conts_role_79M,Type of contact role of a person.
171,empls_economicalst_849M,The economical status of the person (num_group...
172,empls_employedfrom_796D,"Start of employment (num_group1 - person, num_..."
173,empls_employer_name_740M,"Employer's name (num_group1 - person, num_grou..."
415,relatedpersons_role_762T,Relationship type of a client's related person...


In [116]:
#Delete files to save memory
df_items = [key for key, value in globals().items() if key.startswith('train_') and not callable(value)]
for key in df_items:
    del globals()[key]
del df_items, key

### VII. Static Data

In [117]:
# 168-column data: standalone file
train_static0 = pd.read_csv('train/train_static_0_0.csv', low_memory = False)
train_static1 = pd.read_csv('train/train_static_0_1.csv', low_memory = False)
train_static0.head()

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,equalitydataagreement_891L,equalityempfrom_62L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,interestrategrace_34L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcommoditycat_1041M,lastapprcommoditytypec_5251766M,lastapprcredamount_781A,lastapprdate_640D,lastcancelreason_561M,lastdelinqdate_224D,lastdependentsnum_448L,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectdate_50D,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastrepayingdate_696D,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,previouscontdistrict_112M,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
0,0,NaN,NaN,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,30000.0,CAL,0.0,0.0,NaN,NaN,NaN,NaN,0.0,30000.0,GBA,0.0,NaN,0.4500,NaN,NaN,NaN,NaN,0

In [118]:
train_static0.shape

(63032, 168)

In [120]:
train_static0.describe()

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,currdebt_22A,currdebtcredtyperange_828A,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,downpmt_116A,eir_270L,homephncnt_628L,inittransactionamount_650A,interestrate_311L,interestrategrace_34L,lastapprcredamount_781A,lastdependentsnum_448L,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcredamount_222A,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
count,63032.000000,26738.000000,10072.000000,63032.000000,63032.000000,63032.000000,63032.000000,63032.00000,63032.000000,63032.000000,63032.000000,21711.000000,8193.000000,385.000000,23007.000000,21566.000000,384.000000,20268.000000,8785.000000,385.000000,63032.000000,63032.000000,63032.000000,63032.000000,63032.000000,63032.000000,63032.000000,248.000000,63032.000000,63032.000000,63032.000000,63032.000000,63032.000000,63032.000000,63032.000000,63032.000000,23194.000000,22971.000000,33522.0,63032.000000,63032.000000,63032.000000,23211.000000,63032.0,63032.000000,63032.000000,54379.000000,63032.000000,8653.000000,54379.000000,7963.0,23994.000000,2001.000000,100.000000,95.000000,31423.000000,21959.000000,39613.0,39613.0,3.961300e+04,0.0,17495.000000,20845.000000,8593.000000,39613.000000,33521.000000,16305.000000,39613.000000,39613.000000,39613.000000,39613.000000,39613.000000,39613.000000,21565.000000,385.000000,9186.000000,384.000000,21710.000000,385.000000,63031.000000,23210.000000,63031.000000,63031.000000,63031.000000,63031.000000,23201.000000,23764.000000,63031.000000,237

In [121]:
print(f"Total Columns of first file: {len(train_static0.columns)}")
print(f"First vs 2nd File: {sum(train_static0.columns == train_static1.columns)} match")

Total Columns of first file: 168
First vs 2nd File: 168 match


In [122]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_static0.columns)])

,Variable,Description
1,actualdpdtolerance_344P,DPD of client with tolerance.
14,amtinstpaidbefduel24m_4187115A,Number of instalments paid before due date in ...
17,annuity_780A,Monthly annuity amount.
19,annuitynextmonth_57A,Next month's amount of annuity.
20,applicationcnt_361L,Number of applications associated with the sam...
21,applications30d_658L,Number of applications made by the client in t...
22,applicationscnt_1086L,Number of applications associated with the sam...
23,applicationscnt_464L,Number of applications made in the last 30 day...
24,applicationscnt_629L,Number of applications with the same employer ...
25,applicationscnt_867L,Number of applications associated with the sam...


In [123]:
# 53-column data: standalone file
train_static_cb = pd.read_csv('train/train_static_cb_0.csv', low_memory = False)
train_static_cb.head()

,case_id,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
0,357,NaN,NaN,NaN,1988-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,6301.4000,NaN,2019-01-25,NaN,NaN,NaN,NaN,NaN,NaN
1,381,NaN,NaN,NaN,1973-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,4019.6000,NaN,2019-01-25,NaN,NaN,NaN,NaN,NaN,NaN
2,388,NaN,NaN,NaN,1989-04-01,NaN,1989-04-01,NaN,6.0,8.0,2.0,10.0,4.0,a55475b1,a55475b1,a55475b1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,a55475b1,a55475b1,10.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,14548.0000,NaN,2019-01-28,NaN,NaN,NaN,NaN,3.0,5.0
3,405,NaN,NaN,NaN,1974-03-01,NaN,1974-03-01,NaN,0.0,0.0,0.0,1.0,0.0,a55475b1,a55475b1,a55475b1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,a55475b1,a55475b1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,10498.2400,NaN,2019-01-21,NaN,NaN,NaN,NaN,2.0,0.0
4,409,NaN,NaN,NaN,1993-06-01,NaN,1993-06-01,NaN,2.0,3.0,0.0,3.0,1.0,a55475b1,717ddd49,a55475b1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,a7fcb6e5,a55475b1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,6344.8804,NaN,2019-01-21,NaN,NaN,NaN,NaN,0.0,4.0


In [124]:
train_static_cb.shape

(215776, 53)

In [125]:
train_static_cb.describe()

,case_id,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
count,215776.000000,3.247200e+04,200054.000000,200054.000000,200054.000000,200054.000000,200054.000000,200054.000000,6953.000000,6953.000000,6953.000000,6953.0,6953.000000,6953.000000,6953.000000,6953.0,6953.000000,6953.000000,6953.0,6953.000000,6953.0,6953.000000,6953.000000,6953.000000,200054.000000,200054.000000,33428.000000,28690.000000,20907.000000,28690.000000,20907.000000,34960.000000,61184.000000,61184.000000,13823.000000,200054.000000,200054.000000
mean,148900.483636,7.617754e+05,2.196917,3.206074,0.739650,6.281054,1.650014,3.685040,0.000144,5.470013,0.008198,0.0,0.000575,0.339566,0.000719,0.0,0.869409,0.043578,0.0,0.126420,0.0,0.000144,0.001294,3.124119,3.593285,6.281054,8612.906680,9322.722304,17162.328332,6.635378,13.058975,5.787958,6.715792,11987.499934,0.023152,3.417682,3.702995
std,97386.409430,1.024694e+06,2.643653,3.647815,1.148686,6.425162,2.097306,4.393764,0.011993,7.046349,0.091758,0.0,0.023980,0.634101,0.026809,0.0,1.426439,0.204170,0.0,0.336647,0.0,0.011993,0.035957,4.243588,4.144206,6.425162,5006.323023,5120.024988,6542.230338,2.268611,1.852293,1.790264,3.505313,12253.276964,0.971699,3.999336,4.128791
min,357.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.200000,6.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-3.070604,0.000000,0.000000
25%,55146.750000,1.652113e+05,0.000000,1.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,2.000000,6034.067000,6771.400000,13538.200000,6.000000,13.000000,6.000000,5.000000,5085.950075,-0.488776,1.000000,1.000000
50%,155371.500000,4.681123e+05,1.000000,2.000000,0.000000,4.000000,1.000000,2.000000,0.000000,3.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,2.000000,4.000000,7222.200000,7476.300000,15590.400000,6.000000,14.000000,6.000000,6.000000,8689.293000,0.137122,2.000000,2.000000
75%,209316.250000,9.307740e+05,3.000000,4.000000,1.000000,8.000000,2.000000,5.000000,0.000000,8.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,5.000000,5.000000,8.000000,11076.008500,11527.200000,20413.400500,6.000000,14.000000,6.000000,8.000000,15299.450250,0.736941,5.000000,5.000000
max,608444.000000,1.435173e+07,44.000000,57.000000,22.000000,88.000000,41.000000,76.000000,1.000000,57.000000,2.000000,0.0,1.000000,6.000000,1.000000,0.0,14.000000,1.000000,0.0,2.000000,0.0,1.000000,1.000000,37.000000,66.000000,88.000000,68713.500000,205848.610000,99085.400000,15.000000,16.000000,48.000000,121.000000,286638.600000,2.040895,61.000000,62.000000


In [126]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_static_cb.columns)])

,Variable,Description
27,assignmentdate_238D,Tax authority data - date of assignment.
28,assignmentdate_4527235D,Tax authority data - Date of assignment.
29,assignmentdate_4955616D,Tax authority assignment date.
41,birthdate_574D,Client's date of birth (credit bureau data).
84,contractssum_5085716L,Total sum of values of contracts retrieved fro...
119,dateofbirth_337D,Client's date of birth.
120,dateofbirth_342D,Client's date of birth.
126,days120_123L,Number of credit bureau queries for the last 1...
127,days180_256L,Number of credit bureau queries for last 180 d...
128,days30_165L,Number of credit bureau queries for the last 3...


In [127]:
#Delete files to save memory
df_items = [key for key, value in globals().items() if key.startswith('train_') and not callable(value)]
for key in df_items:
    del globals()[key]
del df_items, key

### VIII. Tax Registry

In [131]:
# 5-column data: standalone file
train_taxa = pd.read_csv('train/train_tax_registry_a_1.csv', low_memory = False)
train_taxa.head()

,case_id,amount_4527230A,name_4527232M,num_group1,recorddate_4527225D
0,28631,711.0000,f980a1ea,3.0,2019-09-13
1,28631,1946.0000,f980a1ea,2.0,2019-09-13
2,28631,2600.0000,f980a1ea,1.0,2019-09-13
3,28631,3616.4001,f980a1ea,0.0,2019-09-13
4,28632,400.0000,5f9b74f5,6.0,2019-09-13


In [132]:
train_taxa.describe()

,case_id,amount_4527230A,num_group1
count,957879.000000,957879.000000,957878.000000
mean,567149.067338,2159.301765,3.878713
std,342331.887134,2951.160763,3.683430
min,28631.000000,0.000000,0.000000
25%,190013.000000,850.000000,1.000000
50%,805199.000000,1290.000000,3.000000
75%,855806.000000,2532.600000,5.000000
max,905854.000000,68760.805000,66.000000


In [133]:
train_taxa.shape

(957879, 5)

In [134]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_taxa.columns)])

,Variable,Description
7,amount_4527230A,Tax deductions amount tracked by the governmen...
286,name_4527232M,Name of employer.
409,recorddate_4527225D,Date of tax deduction record.


In [135]:
# 5-column data: standalone file
train_taxb = pd.read_csv('train/train_tax_registry_b_1.csv', low_memory = False)
train_taxb.head()

,case_id,amount_4917619A,deductiondate_4917603D,name_4917606M,num_group1
0,49435,6885.0,2019-10-16,6b730375,0.0
1,49435,6885.0,2019-10-16,6b730375,1.0
2,49435,6885.0,2019-10-16,6b730375,2.0
3,49435,6885.0,2019-10-16,6b730375,3.0
4,49435,6885.0,2019-10-16,6b730375,4.0


In [136]:
train_taxb.shape

(970883, 5)

In [137]:
train_taxb.describe()

,case_id,amount_4917619A,num_group1
count,9.708830e+05,970883.000000,970882.000000
mean,1.345649e+06,20215.938410,4.155527
std,6.505841e+05,25405.979249,4.113286
min,4.943500e+04,0.000000,0.000000
25%,9.911950e+05,6885.000000,1.000000
50%,1.842075e+06,13214.400000,3.000000
75%,1.886777e+06,24364.800000,5.000000
max,1.935239e+06,344250.000000,100.000000


In [142]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_taxb.columns)])

,Variable,Description
8,amount_4917619A,Tax deductions amount tracked by the governmen...
136,deductiondate_4917603D,Tax deduction date.
287,name_4917606M,Name of employer.


In [138]:
# 5-column data: standalone file
train_taxc = pd.read_csv('train/train_tax_registry_c_1.csv', low_memory = False)
train_taxc.head()

,case_id,employername_160M,num_group1,pmtamount_36A,processingdate_168D
0,357,c91b12ff,0,1200.00000,2019-01-04
1,357,c91b12ff,1,1200.00000,2018-11-28
2,357,c91b12ff,2,972.80000,2018-11-01
3,357,c91b12ff,3,628.60004,2018-10-08
4,357,c91b12ff,4,1200.00000,2018-09-10


In [140]:
train_taxc.describe()

,case_id,num_group1,pmtamount_36A
count,1.121984e+06,1.121984e+06,1.121984e+06
mean,4.776740e+05,3.714832e+00,2.058684e+03
std,2.839878e+05,3.722657e+00,2.846579e+03
min,3.570000e+02,0.000000e+00,0.000000e+00
25%,1.399380e+05,1.000000e+00,7.000000e+02
50%,6.418920e+05,3.000000e+00,1.253000e+03
75%,7.016730e+05,5.000000e+00,2.400597e+03
max,7.613170e+05,1.200000e+02,4.250000e+04


In [141]:
train_taxc.shape

(1121984, 5)

In [143]:
with pd.option_context("display.max_rows", 1000):
    display(features[features.Variable.isin(train_taxc.columns)])

,Variable,Description
170,employername_160M,Employer's name.
371,pmtamount_36A,Tax deductions amount for credit bureau payments.
402,processingdate_168D,Date when the tax deduction is processed.
